# 统计方法与机器学习-实验报告2

温兆和 10205501432

## 背景描述

为了调查吃巧克力对心血管健康的影响，实验由三种类型的巧克力组成：100g的黑巧克力，含有200mg全脂牛奶的100g黑巧克力和200g的牛奶巧克力。12个实验对象：7女5男。在不同的天数里，每个实验对象将吃一种类型的巧克力，一个小时后测量他们血浆的总抗氧能力。

## 数据描述

实验次序本身具有随机性，无需再随机化。请使用Project_1.csv中的数据集。数据字典如下表所示。
|变量名|变量含义|变量类型|变量取值范围|
|-----|--------|-------|-----------|
|Chocolate|巧克力类型|分类变量|$\{1,2,3\}$|
|Capacity|血浆浓度|连续变量|$R$|

显著性水平$α$取 0.05。

## 实验过程

- 用Bonferroni方法比较吃了 3 种巧克力后，一个小时的血浆总抗氧能力两两是否存在差异。

在多重比较问题中，$\mu_{i}-\mu_{i'}$落入置信区间$$[(\bar{y_{i.}}-\bar{y_{i'.}})-t_{1-\alpha/2}(n-a)\hat{\sigma}\sqrt{\frac{2}{m}},(\bar{y_{i.}}-\bar{y_{i'.}})+t_{1-\alpha/2}(n-a)\hat{\sigma}\sqrt{\frac{2}{m}}]$$的概率为$1-\alpha$，但所有$\mu_{i}-\mu_{i'}$都落入该置信区间的概率介于$1-a\alpha$和$1-\alpha$之间，这个值可能远小于$1-\alpha$。在Bonferroni方法中，我们把置信水平$1-\alpha$扩大为$1-\frac{\alpha}{a}$，从而保证所有$\mu_{i}-\mu_{i'}$都落入该置信区间的概率至少为$1-\alpha$。

我们首先引入本次实验所需的Python库，并用与上一次实验同样的方法对数据集进行处理，并设定好水平数、样本量、显著性水平等参数：

In [18]:
import os # 修改工作目录

import numpy as np
import pandas as pd
import random
from numpy import random
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scipy.stats as stats # 统计函数
import matplotlib.pyplot as plt
from plotnine import * # ggplot 绘图
from plotnine.data import mpg
from jupyterquiz import display_quiz # Quiz

#from ggplot import ggplot

import math

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from scipy.stats import f
from scipy.stats import t
from statsmodels.stats.stattools import durbin_watson
alpha = 0.05 # significant level
a = 3 # number of levels
m = 12 # number of replicates
n = a*m # sample size
def ChangeNumToLetter(a):
    if a==1:
        return 'A'
    elif a==2:
        return 'B'
    else:
        return 'C'
Data = pd.read_csv("./Project/Project_1.csv")
## Construct a New Dataset
Data = Data[['Chocolate','Capacity']] # select some columns from a dataset
Data.columns = ['chokolate','capacity']
for z in range(0,36):
    Data['chokolate'][z]=ChangeNumToLetter(Data['chokolate'][z])
print(Data)

   chokolate  capacity
0          A     118.8
1          A     122.6
2          A     115.6
3          A     113.6
4          A     119.5
5          A     115.9
6          A     115.8
7          A     115.1
8          A     116.9
9          A     115.4
10         A     115.6
11         A     107.9
12         B     105.4
13         B     101.1
14         B     102.7
15         B      97.1
16         B     101.9
17         B      98.9
18         B     100.0
19         B      99.8
20         B     102.6
21         B     100.9
22         B     104.5
23         B      93.5
24         C     102.1
25         C     105.8
26         C      99.6
27         C     102.7
28         C      98.8
29         C     100.9
30         C     102.8
31         C      98.7
32         C      94.7
33         C      97.8
34         C      99.7
35         C      98.6


/tmp/ipykernel_1457146/2750232432.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


由上周的实验可知，$MS_E=10.4335$。调整显著性水平,并求出三组样本各自的平均值和$t_{1-\alpha/2a}(n-a)$：

In [3]:
alphacomma = alpha/a
MSE = 10.4335
Aaverage = 0
Baverage = 0
Caverage = 0
for z in range(0,36):
    if Data['chokolate'][z]=='A':
        Aaverage+=Data['capacity'][z]
    elif Data['chokolate'][z]=='B':
        Baverage+=Data['capacity'][z]
    else:
        Caverage+=Data['capacity'][z]
Aaverage = Aaverage/m
Baverage = Baverage/m
Caverage = Caverage/m
td=round(t.ppf(1-alphacomma/2,n-a),4)

用调整后的显著性水平比较A、B两种巧克力对心血管健康的影响：

In [4]:
lower1 = Aaverage-Baverage-td*MSE*((2/m)**0.5)
upper1 = Aaverage-Baverage+td*MSE*((2/m)**0.5)
print(lower1)
print(upper1)

4.615127007390646
26.10153965927605


可见，0并没有落入置信区间。我们拒绝原假设，认为A、B两种巧克力对心血管健康的影响不同。

下面，我们用同样的方法比较B、C两种巧克力对心血管健康的影响：

In [5]:
lower2 = Baverage-Caverage-td*MSE*((2/m)**0.5)
upper2 = Baverage-Caverage+td*MSE*((2/m)**0.5)
print(lower2)
print(upper2)

-10.226539659276037
11.259872992609369


可见，0落入了置信区间。我们接受原假设，认为B、C两种巧克力对心血管健康的影响不同。

下面，我们用同样的方法比较A、C两种巧克力对心血管健康的影响：

In [6]:
lower3 = Aaverage-Caverage-td*MSE*((2/m)**0.5)
upper3 = Aaverage-Caverage+td*MSE*((2/m)**0.5)
print(lower3)
print(upper3)

5.131793674057311
26.618206325942715


可见，0并没有落入置信区间。我们拒绝原假设，认为A、C两种巧克力对心血管健康的影响不同。

- 用Tukey方法比较吃了 3 种巧克力后，一个小时的血浆总抗氧能力两两是否存在差异。采用以下两种不同的方法来解决这个问题：
    
    1.直接调用python中现有函数；
    
    2.用蒙特卡洛随机模拟分布的方式，确定$t$化极差统计量的分位数$q_{1-α}(a,df)$,计算临界值(critical value)$c=q_{1-α}(a,df)\frac{\hat{\sigma}}{m}$。

在多重比较问题中，我们同时比较所有的平均值组合。这其中至少一对平均值组合不相等就构成了拒绝域$$W=\bigcup_{1\leq i<i'\leq a}\{|\bar{y_{i.}}-\bar{y_{i'.}}|\geq c_{ii'}\}$$

为了简化问题，我们假设$$对\forall i,i'\in\{1\leq i<i'\leq a\},c_{ii'}=c$$

由于在原假设成立时诸$\mu_{i}$都等于$\mu$，易得$$P(W)=P(max_{i}\frac{\bar{y_{i.}}-\mu}{\frac{\hat{\sigma}}{\sqrt{m}}}-min_{i}\frac{\bar{y_{i.}}-\mu}{\frac{\hat{\sigma}}{\sqrt{m}}}\geq\frac{c}{\frac{\hat{\sigma}}{\sqrt{m}}})$$

令$$q(a,df)=max_{i}\frac{\bar{y_{i.}}-\mu}{\frac{\hat{\sigma}}{\sqrt{m}}}-min_{i}\frac{\bar{y_{i.}}-\mu}{\frac{\hat{\sigma}} {\sqrt{m}}},df=n-a$$

由于$$\frac{\bar{y_{i.}}-\mu}{\frac{\hat{\sigma}}{\sqrt{m}}}\sim t(n-a)$$

所以我们可以把$q(a,df)$看成$a$个独立同分布的自由度为$df$的$t$分布的随机变量的极差。

我们先用Python内置的函数来通过Tukey方法对数据集Data进行检验。

In [7]:
Tukey= pairwise_tukeyhsd(endog = Data["capacity"], groups=Data["chokolate"],alpha=alpha)
print(Tukey)

 Multiple Comparison of Means - Tukey HSD, FWER=0.05  
group1 group2 meandiff p-adj   lower    upper   reject
------------------------------------------------------
     A      B -15.3583    0.0 -18.5941 -12.1226   True
     A      C  -15.875    0.0 -19.1108 -12.6392   True
     B      C  -0.5167 0.9191  -3.7524   2.7191  False
------------------------------------------------------


可见，A巧克力与B巧克力、A巧克力与C巧克力对心血管健康的影响不同，而B巧克力与C巧克力对心血管健康的影响相同。

接着，我们再使用蒙特卡洛方法得到$q(a,df)$的分布。在显著性水平$\alpha$下，由于$$P(W)=P(q(a,df)\geq\frac{c}{\frac{\hat{\sigma}}{\sqrt{m}}})=\alpha$$

我们可以得到$$c=q_{1-\alpha}(a,df)\frac{\hat{\sigma}}{\sqrt{m}}$$

我们只要将不同水平下的样本均值两两作差并与临界值$c$比较，就可以做出判断了。

在本实验中，水平数$a=3$，$t$分布的自由度为$n-a=33$，$\bar{\sigma}=MS_E=10.4335$，我们令重复次数$N=1000000$。

In [24]:
Q_N = []
for z in range(0,1000000):
    N_01 = [random.randn(),random.randn(),random.randn()]
    N_01.sort()
    chi = random.chisquare(n-a)
    chi = chi/(n-a)
    q_n = (N_01[2]-N_01[0])/(chi**0.5)
    Q_N.append(q_n)
Q_N.sort()
# 1-α分位数为950000，方差的估计MSE=10.4335
c_aim = Q_N[950000]*(10.4335/(m**0.5))
print(c_aim)

10.442934601227966


这时，我们就得到了这个多重比较问题拒绝域中的临界值$c$。下面，我们对三组样本的样本均值两两做差并与刚刚得到的临界值进行比较，并得出结论。

In [27]:
delta_AB = Aaverage - Baverage
delta_BC = Baverage - Caverage
delta_AC = Aaverage - Caverage
print(delta_AB)
print(delta_BC)
print(delta_AC)

15.358333333333348
0.5166666666666657
15.875000000000014


可见，巧克力A与巧克力B、巧克力A与巧克力C的样本均值之差大于我们刚才给定的临界值10.4429，所以我们拒绝原假设，认为巧克力A与巧克力B、巧克力A与巧克力C对心血管健康的影响不一样；而巧克力B与巧克力C的样本均值之差小于这个临界值，所以我们接受原假设，认为巧克力B与巧克力C对心血管健康的影响一样。这个结果与我们刚刚用Python内置的Tukey方法函数得到的结果是一致的。

- 基于这个例子，请评述一下Bonferroni方法和Tukey方法的异同。

同：在这个实验中，Bonferroni方法和Tukey方法都是想在一整个数据集上（也就是巧克力A、B、C对心血管健康的影响）比较两个不同水平下的样本均值是否相等（也就是比较两种不同的巧克力对心血管健康的影响的异同）。当水平数为$a$时，我们要计算$O(a^2)$个拒绝域，但当原假设成立（三种巧克力对心血管健康影响一致）时，每一对水平的均值之差都落入接受域的概率会小于原有的显著性水平$1-\alpha$。这两种方法也都是为了解决这个问题。

异：Bonferroni方法先确定当原假设成立时，每一对水平的均值之差都落入接受域的概率介于$1-a\alpha$和$1-\alpha$之间，再把单对水平均值差（两种巧克力对心血管健康影响的均值之差）区间估计的置信水平升为$1-\frac{\alpha}{a}$，从而保证当原假设成立时，每一对水平的均值之差都落入接受域的概率为$1-\alpha$。但这会导致单对水平均值差的置信区间变大，精度降低；

Tukey方法则是将每一对水平（每两种巧克力）比较时拒绝域的临界值都设定为同一个值$c$，并证明当原假设成立时$c$与$t$分布极差统计量的分位数有关，再求$t$分布极差统计量的$1-\alpha$分位数，从而保证当原假设成立（三种巧克力对心血管健康影响一致）时，每一对水平的均值之差都落入接受域的概率等于$1-\alpha$。